In [ ]:
%pip install openai langchain langchain_openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain.chains.sequential import SequentialChain
import os

In [ ]:
# Setup the chat model:
os.envirion['OPENAI_API_KEY'] = 'API_KEY'
chat = ChatOpenAI()  # type: ignore

In [ ]:
# Create the article outline generation chain:
template = """You are responsible for creating a very detailed article outline for {blog_post_idea}"""
article_outline_generation_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(template),
    output_key="article_outline",
)

In [ ]:
# Create the article generation chain:
template = """Given an article outline, write a very high quality and detailed article.
    The article must be at least {word_count} words in length.
    Article outline: {article_outline}
    """
article_generation_chain = LLMChain(
    llm=chat,
    prompt=PromptTemplate.from_template(template),
    output_key="generated_article",
)

In [ ]:
# Create the article editing chain:
template = """Given an article, edit it to be very high quality and detailed.
    The article must be at least {word_count} words in length.

    You must follow the following principles:
    1. The article must be very detailed and high quality.
    2. If the article is under {word_count} words, you must add more content to it.
    3. If you see any grammatical errors, you must fix them.
    4. If you see any spelling errors, you must fix them.
    5. If you see any factual errors, you must fix them.

    Here is the article: {generated_article}
    """
article_editing_chain = LLMChain(
    llm=chat, prompt=PromptTemplate.from_template(template), output_key="edited_article"
)

In [ ]:
# Combine three chains into a single chain:
chain = SequentialChain(
    chains=[
        article_outline_generation_chain,
        article_generation_chain,
        article_editing_chain,
    ],
    input_variables=["blog_post_idea", "word_count"],
    output_variables=["article_outline", "generated_article", "edited_article"],
    verbose=True,
)

In [ ]:
# Run the chain:
article_results = chain(
    {
        "blog_post_idea": "How to become a better data engineer.",
        "word_count": 1000,
    },
)

In [ ]:
article_results["edited_article"]

"Title: The Impact of Artificial Intelligence on Modern Society: A Comprehensive Analysis\n\nIntroduction:\n\nIn the rapidly evolving landscape of technology, one phenomenon has emerged as a transformative force, revolutionizing industries and reshaping the way we live and work: Artificial Intelligence (AI). With its ability to process vast amounts of data, make complex decisions, and learn from experience, AI has become a powerful tool with immense potential. This article aims to provide a detailed examination of the impact of AI on modern society, delving into its applications across various sectors, discussing ethical considerations, and exploring the future implications.\n\nI. Understanding Artificial Intelligence:\n\nArtificial Intelligence refers to the development of intelligent machines capable of performing tasks typically requiring human intelligence. These machines are designed to exhibit behaviors that imitate, and in some cases surpass, human cognitive abilities. This enco

# Translating The Article To English:


In [ ]:
from __future__ import annotations
from typing import Any, Dict, List, Optional

from pydantic import Extra
from langchain.callbacks.manager import (
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate


class MyCustomChain(Chain):
    """
    An example of a custom chain.
    """

    prompt: BasePromptTemplate
    """Prompt object to use."""
    llm: Any
    output_key: str = "text"  #: :meta private:
    languages: List[str] = ["French", "Spanish", "German"]  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

        extra = Extra.forbid
        arbitrary_types_allowed = True

    @property
    def input_keys(self) -> List[str]:
        """Will be whatever keys the prompt expects.

        :meta private:
        """
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Will always return text key.

        :meta private:
        """
        return [self.output_key]

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        # The custom chain logic goes here:
        translations = {}
        for language in self.languages:
            prompt_value = self.prompt.format_prompt(
                **{"language": language, "edited_article": inputs["edited_article"]}
            )
            translations[language] = self.llm.generate_prompt(
                [prompt_value],
                callbacks=run_manager.get_child() if run_manager else None,
            )

        return {self.output_key: translations}  # type: ignore

    @property
    def _chain_type(self) -> str:
        return "my_custom_chain"

In [ ]:
chain = MyCustomChain(
    prompt=PromptTemplate.from_template(
        """You are receiving an article and must translate it into {language}.\n\n{edited_article}"""
    ),
    llm=ChatOpenAI(),  # type: ignore
    output_key="translations",
)

result = chain({"edited_article": "I love learning programming!", "language": "French"})

In [ ]:
print(result["translations"]["French"].generations[0][0].text)
print(result["translations"]["Spanish"].generations[0][0].text)
print(result["translations"]["German"].generations[0][0].text)

J'adore apprendre la programmation !
¡Me encanta aprender programación!
Ich liebe es, Programmieren zu lernen!


---

Now you can easily add the custom translation chain to your previous `SequentialChain` object:


In [ ]:
translation_sequential_chain = SequentialChain(
    chains=[
        article_outline_generation_chain,
        article_generation_chain,
        article_editing_chain,
        chain,
    ],
    input_variables=["blog_post_idea", "word_count", "language"],
    output_variables=[
        "article_outline",
        "generated_article",
        "edited_article",
        "translations",
    ],
    verbose=True,
)

In [ ]:
translation_chain_results = translation_sequential_chain(
    {
        "blog_post_idea": "How to become a better data engineer.",
        "word_count": 1000,
        "language": "",  # Dummy key here as you are accessing the languages via self.languages on the custom LLM chain:
    },
)



> Entering new  chain...

> Finished chain.


In [ ]:
translation_chain_results

{'blog_post_idea': 'How to become a better data engineer.',
 'word_count': 1000,
 'language': '',
 'article_outline': "I. Introduction\n    A. Importance of data engineering in today's world\n    B. Overview of the role and responsibilities of a data engineer\n    C. Benefits of becoming a better data engineer\n\nII. Understanding the Fundamentals\n    A. Familiarizing with core data engineering concepts\n        1. Definition of data engineering\n        2. Key terms and technologies\n        3. Differences between data engineering, data science, and data analysis\n    B. Developing strong coding skills\n        1. Preferred programming languages for data engineering (Python, SQL, etc.)\n        2. Best practices for writing efficient and scalable code\n        3. Understanding data structures and algorithms\n\nIII. Mastering Data Storage and Processing\n    A. Learning about different data storage systems\n        1. Relational databases (e.g., MySQL, PostgreSQL)\n        2. NoSQL da